# TODO
early_amazon

# SGCN用データセット作り

In [ ]:
import numpy as np
import pandas as pd
import pickle
import networkx as nx
import matplotlib
%matplotlib inline

In [ ]:
def get_dist(df,col,rating_cols=None):
    df_cnt = df.groupby([col]+['rating'])['time'].count().unstack(1,fill_value=0)
    df_dist = pd.DataFrame(df_cnt.values / df_cnt.sum(1).values.reshape(-1,1),
                                               columns=df_cnt.columns,
                                               index=df_cnt.index)
    if rating_cols is not None:
        df_dist = pd.concat([pd.DataFrame(columns=rating_cols),df_dist]).fillna(0)
    return df_dist

## Amazon → SGCN

### user-product network

#### network

In [ ]:
amazon_dir = 'amazon'

In [ ]:
amazon_network = pd.read_csv('raw_data/{0}/{0}_network.csv'.format(amazon_dir),header=None)
amazon_network.columns = ['user_id','product_id','rating','time']
amazon_network['weight'] = amazon_network.rating.map(lambda x:(x-3)/2).round()

amazon_gt = pd.read_csv('raw_data/{0}/{0}_gt.csv'.format(amazon_dir),header=None)
amazon_gt.columns = ['user_id','label']

In [ ]:
truncated_amazon_network = amazon_network.loc[amazon_network.weight!=0,['user_id','product_id','weight']]

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(np.hstack((truncated_amazon_network.user_id,
                                                   truncated_amazon_network.product_id,
                                                   amazon_gt.user_id)))

In [ ]:
truncated_amazon_network['id1'] = label_encoder.transform(truncated_amazon_network.user_id)

truncated_amazon_network['id2'] = label_encoder.transform(truncated_amazon_network.product_id)

amazon_gt['node_id'] = label_encoder.transform(amazon_gt.user_id)

#### node features

In [ ]:
user_dist = get_dist(amazon_network,'user_id')

product_dist = get_dist(amazon_network,'product_id')

# user_product_dist = user_dist.append(product_dist)

user_product_dist = pd.concat([user_dist,product_dist],1).fillna(0)

In [ ]:
node_features_df = user_product_dist.loc[label_encoder.classes_]

#### ファイル出力

In [ ]:
truncated_amazon_network[['id1','id2','weight']].to_csv('input/{0}/{0}_network.csv'.format(amazon_dir),index=None)

amazon_gt[['node_id','label']].to_csv('input/{0}/{0}_gt.csv'.format(amazon_dir),index=None)

np.save(arr=label_encoder.classes_,file='input/{0}/{0}_label_encoder.npy'.format(amazon_dir))

node_features_df.to_csv('input/{0}/{0}_node_feature.csv'.format(amazon_dir),index=None)

## epinions

### network

In [ ]:
epinions_network = pd.read_csv('raw_data/epinions/epinions_network.csv',header=None)

epinions_network.columns = ['id1','id2','rating','time']

epinions_network['weight'] = epinions_network.rating.map(lambda x:-1 if x-3.5 < 0 else 1)

In [ ]:
epinions_gt = pd.read_csv('raw_data/epinions/epinions_gt.csv',header=None)

epinions_gt.columns = ['user_id','label']

In [ ]:
epinions_gt = epinions_gt.loc[epinions_gt.user_id.isin(set(epinions_network.id1.unique()) | set(epinions_network.id2.unique()))]

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(np.hstack((epinions_network.id1,
                                                   epinions_network.id2,
                                                   epinions_gt.user_id)))

In [ ]:
epinions_network['id1_'] = label_encoder.transform(epinions_network.id1)

epinions_network['id2_'] = label_encoder.transform(epinions_network.id2)

epinions_gt['node_id'] = label_encoder.transform(epinions_gt.user_id)

### node features

In [ ]:
node_features_df = pd.concat([get_dist(epinions_network,'id1_'),get_dist(epinions_network,'id2_')],1).fillna(0).sort_index()

### ファイル出力

In [ ]:
epinions_network[['id1_','id2_','weight']].to_csv('input/epinions/epinions_network.csv',index=None)
epinions_gt[['node_id','label']].to_csv('input/epinions/epinions_gt.csv',index=None)
np.save(arr=label_encoder.classes_,file='input/epinions/epinions_label_encoder.npy')
node_features_df.to_csv('input/epinions/epinions_node_feature.csv',index=None)

## epinions_sub

In [ ]:
sampled_nodes = np.random.choice()

## alpha

In [ ]:
alpha_network = pd.read_csv('raw_data/alpha/alpha_network.csv',header=None)

alpha_network.columns = ['id1','id2','rating','time']

alpha_network['weight'] = alpha_network.rating.map(lambda x:1 if x>0 else -1)

In [ ]:
pd.to_datetime(alpha_network.time,unit='s').hist()

In [ ]:
alpha_gt = pd.read_csv('raw_data/alpha/alpha_gt.csv',header=None)

alpha_gt.columns = ['user_id','label']

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(np.hstack((alpha_network.id1,
                                                   alpha_network.id2,
                                                   alpha_gt.user_id)))

In [ ]:
alpha_network['id1_'] = label_encoder.transform(alpha_network.id1)

alpha_network['id2_'] = label_encoder.transform(alpha_network.id2)

alpha_gt['node_id'] = label_encoder.transform(alpha_gt.user_id)

node_features_df = pd.concat([get_dist(alpha_network,'id1_'),get_dist(alpha_network,'id2_')],1).fillna(0).sort_index()

In [ ]:
alpha_network[['id1_','id2_','weight']].to_csv('input/alpha/alpha_network.csv',index=None)
alpha_gt[['node_id','label']].to_csv('input/alpha/alpha_gt.csv',index=None)
np.save(arr=label_encoder.classes_,file='input/alpha/alpha_label_encoder.npy')
node_features_df.to_csv('input/alpha/alpha_node_feature.csv',index=None)

## otc

In [ ]:
otc_network = pd.read_csv('raw_data/otc/otc_network.csv',header=None)

otc_network.columns = ['id1','id2','rating','time']

otc_network['weight'] = otc_network.rating.map(lambda x:1 if x>0 else -1)

In [ ]:
otc_gt = pd.read_csv('raw_data/otc/otc_gt.csv',header=None)

otc_gt.columns = ['user_id','label']

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(np.hstack((otc_network.id1,
                                                   otc_network.id2,
                                                   otc_gt.user_id)))

In [ ]:
otc_network['id1_'] = label_encoder.transform(otc_network.id1)

otc_network['id2_'] = label_encoder.transform(otc_network.id2)

otc_gt['node_id'] = label_encoder.transform(otc_gt.user_id)

node_features_df = pd.concat([get_dist(otc_network,'id1_'),get_dist(otc_network,'id2_')],1).fillna(0).sort_index()

In [ ]:
otc_network[['id1_','id2_','weight']].to_csv('input/otc/otc_network.csv',index=None)
otc_gt[['node_id','label']].to_csv('input/otc/otc_gt.csv',index=None)
np.save(arr=label_encoder.classes_,file='input/otc/otc_label_encoder.npy')
node_features_df.to_csv('input/otc/otc_node_feature.csv',index=None)

## 最初のr割だけを取ってくる

In [ ]:
import os
from sklearn.preprocessing import LabelEncoder

In [ ]:
def make_early_amazon(data_name,rate=0.1):
    df_ = pd.read_csv(f'raw_data/{data_name}/{data_name}_network.csv',header=None)
    df_.columns = ['id1','id2','rating','time']
    df_['weight'] = df_.rating.map(lambda x:(x-3)/2).round()
    
    
    # ここで最初の1割だけを切り取る
    df = df_.sort_values('time').iloc[:int(len(df_)*rate),:]
    trunc_df = df.loc[df.weight!=0]
    
    gt_df = pd.read_csv(f'raw_data/{data_name}/{data_name}_gt.csv',header=None)
    gt_df.columns = ['user_id','label']    
    
    # gtの方も切り取る
    gt_df = gt_df.loc[gt_df.user_id.isin(set(trunc_df.id1) | set(trunc_df.id2))]
    
    label_encoder = LabelEncoder()
    label_encoder.fit(np.hstack((trunc_df.id1,trunc_df.id2,gt_df.user_id)))
    
    trunc_df['id1_'] = label_encoder.transform(trunc_df.id1)
    trunc_df['id2_'] = label_encoder.transform(trunc_df.id2)
    gt_df['node_id'] = label_encoder.transform(gt_df.user_id)
    
    
    early_node_features_df = pd.concat([get_dist(df_,'id1'),
                                                                       get_dist(df_,'id2')],1).fillna(0).sort_index()
    early_node_features_df = early_node_features_df.loc[label_encoder.classes_]
    
    # assert f'{data_name}_early' not in os.listdir('input'), "Directory already exists"
    # assert gt_df['label'].value_counts().min()>=10, "Cannot do 10-fold cross validation"
    print(gt_df.label.value_counts())
    os.mkdir(f'input/{data_name}_early{rate}')
    trunc_df[['id1_','id2_','weight']].to_csv(f'input/{data_name}_early{rate}/{data_name}_early{rate}_network.csv',index=None)
    gt_df[['node_id','label']].to_csv(f'input/{data_name}_early{rate}/{data_name}_early{rate}_gt.csv',index=None)
    np.save(arr=label_encoder.classes_,file=f'input/{data_name}_early{rate}/{data_name}_early{rate}_label_encoder.npy')
    early_node_features_df.to_csv(f'input/{data_name}_early{rate}/{data_name}_early{rate}_node_feature.csv',index=None)    
    
    
    return trunc_df, gt_df, early_node_features_df, label_encoder

In [ ]:
def make_early_network(data_name,rate=0.1):
    df_ = pd.read_csv(f'raw_data/{data_name}/{data_name}_network.csv',header=None)
    df_.columns = ['id1','id2','rating','time']
    if (data_name=='otc') or (data_name=='alpha'):
        df_['weight'] = df_.rating.map(lambda x:1 if x>0 else -1)
    elif data_name == 'epinions':
        df_['weight'] = df_.rating.map(lambda x:-1 if x-3.5 < 0 else 1)
    elif data_name == 'amazon':
        return make_early_amazon(data_name,rate=rate)
    else:
        assert False, 'Not Supported'
    # ここで最初の1割だけを切り取る
    df = df_.sort_values('time').iloc[:int(len(df_)*rate),:]
    
    gt_df = pd.read_csv(f'raw_data/{data_name}/{data_name}_gt.csv',header=None)
    gt_df.columns = ['user_id','label']    
    
    # gtの方も切り取る
    gt_df = gt_df.loc[gt_df.user_id.isin(set(df.id1) | set(df.id2))]
    
    label_encoder = LabelEncoder()
    label_encoder.fit(np.hstack((df.id1,df.id2,gt_df.user_id)))
    
    df['id1_'] = label_encoder.transform(df.id1)
    df['id2_'] = label_encoder.transform(df.id2)
    gt_df['node_id'] = label_encoder.transform(gt_df.user_id)
    
    
    all_node_features_id1 = get_dist(df_,'id1')
    all_node_features_id2 = get_dist(df_,'id2')
    early_node_features_df = pd.concat([get_dist(df,'id1_',rating_cols=all_node_features_id1.columns),
                                                                       get_dist(df,'id2_',rating_cols=all_node_features_id2.columns)],1).fillna(0).sort_index()    
    print(gt_df.label.value_counts())
    # assert f'{data_name}_early' not in os.listdir('input'), "Directory already exists"
    # assert gt_df['label'].value_counts().min()>=10, "Cannot do 10-fold cross validation"
    os.mkdir(f'input/{data_name}_early{rate}')
    df[['id1_','id2_','weight']].to_csv(f'input/{data_name}_early{rate}/{data_name}_early{rate}_network.csv',index=None)
    gt_df[['node_id','label']].to_csv(f'input/{data_name}_early{rate}/{data_name}_early{rate}_gt.csv',index=None)
    np.save(arr=label_encoder.classes_,file=f'input/{data_name}_early{rate}/{data_name}_early{rate}_label_encoder.npy')
    early_node_features_df.to_csv(f'input/{data_name}_early{rate}/{data_name}_early{rate}_node_feature.csv',index=None)    
    
    
    return df, gt_df, early_node_features_df, label_encoder

In [ ]:
early_network, early_gt, early_node_features, early_label_encoder = make_early_network('epinions',0.007)

## Amazon Extra

```python
file_path = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz'
review_df_raw = pd.read_json(file_path,lines=True)
assert ~review_df_raw.duplicated(['asin','reviewerID']).any()

def preprocessing(df):
    df_ = df.copy()
    # converting 'reviewTime' column to datetime
    df_['reviewTime'] = pd.to_datetime(df_.reviewTime,format='%m %d, %Y')
    # vote_sum and helpful rate, helpful_bin
    df_['vote_sum'] = df_['helpful'].map(lambda x:x[1])
    df_['helpful_rate'] = df_['helpful'].map(lambda x:x[0]/x[1] if x[1]>0 else float('nan'))
    df_['helpful_bin'] = pd.cut(df_.helpful_rate,bins=np.arange(0,1.1,0.1),include_lowest=True,labels=range(10))    
    # おかしいデータを取り除く
    df_ = df_.loc[~(df_.helpful_rate>1.0)]
    return df_

review_df = preprocessing(review_df_raw)

def generate_network_csv(df, from_date):
    review_df_from_ = df.loc[df.reviewTime>=from_date]
    return review_df_from_[['reviewerID','asin','overall','reviewTime']]

def generate_gt(df,from_date):
    reviewer_all_votes = \
        df.loc[df.reviewTime>=from_date].groupby('reviewerID',as_index=False)['helpful'].agg(lambda x:list(np.vstack(x).sum(0)))

    reviewer_all_votes['vote_sum'] = reviewer_all_votes.helpful.map(lambda x:x[1])

    reviewer_all_votes['rate'] = reviewer_all_votes.helpful.map(lambda x:x[0]/x[1])
    selected_df = reviewer_all_votes.loc[(reviewer_all_votes.vote_sum>=50) &
                                         ((reviewer_all_votes.rate<=0.25) | 
                                          (reviewer_all_votes.rate>=0.75))]
    selected_df['label'] = selected_df['rate'].map(lambda x: -1 if x <= 0.25 else 1)
    
    return selected_df[['reviewerID','label']].set_index('reviewerID')

amazon_network = generate_network_csv(review_df,pd.Timestamp(2013,1,1))

amazon_gt = generate_gt(review_df,pd.Timestamp(2013,1,1))
```

# Appindix

## amazon user networkを作る

In [ ]:
amazon_network

In [ ]:
self_joined = pd.merge(amazon_network,amazon_network,on='product_id',how='right')

self_joined = self_joined.loc[~(self_joined.user_id_x==self_joined.user_id_y)]

self_joined['sign'] = self_joined.weight_x*self_joined.weight_y

In [ ]:
user_network = self_joined.loc[self_joined.sign!=0,['user_id_x','user_id_y','sign']]

user_network = user_network.groupby(['user_id_x','user_id_y'],as_index=False)['sign'].mean().round()

user_network = user_network.loc[user_network.sign!=0]

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(np.hstack((user_network.user_id_x,
                                                   user_network.user_id_y,
                                                   amazon_gt.user_id)))

In [ ]:
user_network['id1'] = label_encoder.transform(user_network.user_id_x)

user_network['id2'] = label_encoder.transform(user_network.user_id_y)

In [ ]:
for_nx_network = user_network.copy()[['id1','id2','sign']]

for_nx_network.columns = ['source','target','weight']

G = nx.from_pandas_edgelist(for_nx_network,edge_attr=True)

In [ ]:
amazon_user_network = nx.to_pandas_edgelist(G)

In [ ]:
amazon_gt['node_id'] = label_encoder.transform(amazon_gt.user_id)

In [ ]:
amazon_user_network[['source','target','weight']].to_csv('input/amazon/user_network.csv',index=None)

amazon_gt[['node_id','label']].to_csv('input/amazon/user_gt.csv',index=None)

np.save(arr=label_encoder.classes_,file='input/amazon/user_label_encoder.npy')